In [1]:
%load_ext autoreload
%autoreload 2

In [53]:
from plaid.diffusion import GaussianDiffusion
from plaid.denoisers import SimpleDiT
from plaid.compression import UncompressContinuousLatent
import torch

# 5x21u86p

ckpt_path = "/homefs/home/lux70/storage/plaid/checkpoints/diffusion/5x21u86p/epoch=5687-step=1200000.ckpt"
ckpt = torch.load(ckpt_path)



denoiser = SimpleDiT(
    depth=5,
    hidden_size=1152,
    input_dim=8,
    max_seq_len=512,
    mlp_ratio=4,
    num_heads=18,
    use_self_conditioning=False
)
# diffusion = GaussianDiffusion(denoiser)
# diffusion = GaussianDiffusion.load_from_checkpoint(ckpt_path, model=denoiser)

state_dict = ckpt['state_dict']
new_sd = {}
for k, v in state_dict.items():
    new_sd[k.lstrip("model._orig_mod.")] = v

denoiser.load_state_dict(new_sd)

ImportError: cannot import name 'UncompressContinuousLatent' from 'plaid.compression' (unknown location)

In [54]:
from plaid.compression.uncompress import UncompressContinuousLatent
uncompressor = UncompressContinuousLatent("jzlv54wl")

using quantizer tanh


In [55]:
diffusion=GaussianDiffusion(
    model=denoiser,
    sampling_timesteps=1000,
    sampling_seq_len=512,
    uncompressor = uncompressor
)

In [67]:
from plaid.callbacks import SampleCallback
denoiser = denoiser.to(device)
diffusion = diffusion.to(device)

callback = SampleCallback(
    diffusion=diffusion,
    save_generated_structures=True,
    batch_size=4,
    num_recycles=2
)


In [44]:
device = torch.device("cuda")
x, log_dict = callback.sample_compressed_latent((64, 512, 8))

  0%|                                                                                                 | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [05:59<00:00, 359.22s/it]


In [56]:
latent = diffusion.process_x_to_latent(x)  # (N, L, C_comp) -> (N, L, C_lat)
print(x.shape)
print(latent.shape)

torch.Size([64, 512, 8])
torch.Size([64, 512, 1024])


In [57]:
seq_str, log_dict = callback.construct_sequence(latent, device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Mean perplexity: 21.749


In [ ]:
pdb_strs, metrics, log_dict = callback.construct_structure(latent[:256], seq_str[:256], device)
    # if log_to_wandb:
    #     logger.log(log_dict)

    # if self.save_generated_structures:
    #     all_pdb_paths = self.save_structures_to_disk(pdb_strs, pl_module.global_step)

    #     if not self.n_structures_to_log is None:
    #         df = pd.DataFrame(metrics)
    #         df['pdb_path'] = [str(x) for x in all_pdb_paths]
    #         df = df.sort_values(by="plddt", ascending=False) 
    #         df = df.iloc[:self.n_structures_to_log, :]
    #         df['protein'] = df['pdb_path'].map(lambda x: wandb.Molecule(str(x)))
    #         wandb.log({"sampled_proteins": wandb.Table(dataframe=df)})


loading esmfold model...
Creating ESMFold...
ESMFold model loaded in 0.52 minutes.
ESMFold model created in 32.01 seconds.


(Generating structure):  50%|████████████████████████████████                                | 8/16 [07:50<07:50, 58.84s/it]

In [74]:
from plaid.utils import write_pdb_to_disk

for i, pdbstr in enumerate(pdb_strs):
    write_pdb_to_disk(pdbstr, f"/homefs/home/lux70/storage/plaid/artifacts/notebook_samples/5x21u86p/{i}.pdb")